In [1]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.lines import Line2D
from pylab import rcParams
rcParams['figure.figsize'] = 25, 10
import numpy as np 
import dill as pickle
import os
import random
import datetime 
import sys 
import seaborn as sns
from einops import rearrange

import matplotlib as mpl
from matplotlib.patches import Rectangle
mpl.rcParams['text.usetex'] = True
mpl.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}'] #for \text command

np.random.seed(0)
random.seed(0)
now = datetime.datetime.now().strftime("%B_%d_%Y_%H_%M_%S")
workingdirectory = os.popen('git rev-parse --show-toplevel').read()[:-1]
sys.path.append(workingdirectory)
os.chdir(workingdirectory)


from codes.experimentclasses.AtomicRegression2 import AtomicRegression
from codes.geometer.RiemannianManifold import RiemannianManifold
from codes.otherfunctions.data_stream import data_stream
from codes.geometer.ShapeSpace import compute3angles
from codes.otherfunctions.get_dictionaries import get_atoms_4
import numpy as np
import torch
import math
from sklearn.decomposition import TruncatedSVD



class RigidEthanolXYZ(AtomicRegression):
    """
    This class estimates
    Parameters
    ----------
    filename : string,
        Data file to load
    ii : np.array(dtype = int),
        List of adjacencies
    jj : np.array,
        List of adjacencies part 2
    d : int,
        dimension over which to evaluate the radii (smaller usually better)
    rmin : float,
        smallest radius ( = rad_bw_ratio * bandwidth) to consider
    rmax : float,
        largest radius ( = rad_bw_ratio * bandwidth) to consider
    ntry : int,
        number of radii between rmax and rmin to try
    run_parallel : bool,
        whether to run the analysis in parallel over radii
    search_space : str,
        either 'linspace' or 'logspace', choose to search in log or linear space
    rad_bw_ratio : float,
        the ratio of radius and kernel bandwidth, default to be 3 (radius = 3*h)
    Methods
    -------
    load_data :
        Loads filename as AtomicRegression.data
    get_atoms_4 :
    	Gets atomic tetrahedra based off of ii and jj
    get_atoms_3 :
    	Gets triples of atoms

    """

    # AtomicRegression(dim, ii, jj, filename)
    def __init__(self, dim, cor, xvar, cores, noise, custom_bonds=None ):
        # def __init__(self, r, R, p,n,d, selectedpoints, dim):
        # self.ii = ii
        # self.jj = jj
        self.n = 10000
        self.natoms = 9
        self.cor = cor
        self.cores = cores
        self.noise = noise
        # n_atoms = len(np.unique(ii))
        self.xvar = xvar
        #self.atoms4, self.p = self.get_atoms_4()
        self.d = 27
        # self.atoms3, self.d = self.get_atoms_3(natoms)
        # self.selectedpoints = selectedpoints
        self.dim = dim
        if custom_bonds.any() != None:
            self.atoms4 = custom_bonds
            self.p = custom_bonds.shape[0]
        # self.n = n

    def load_data(self, angles=False):
        # filename = 'tolueneangles.npz'
        n = self.n
        d = self.d
        dim = self.dim
        cor = self.cor
        natoms = self.natoms
        xvar = self.xvar
        cores = self.cores

        positions = np.zeros((n, natoms, 3))
        positions[0,0,:] = np.asarray([0.,0.,0.])
        positions[0,1,:] = np.asarray([-10.,0.,np.sqrt(2)/100])
        positions[0,2,:] = np.asarray([0.,10.,np.sqrt(3)/100])
        #positions[0,8,:] = np.asarray([1.,1.,0.])
        positions[0,8,:] = np.asarray([1.,10.,np.sqrt(5)/100])
        positions[0,3,:] = np.asarray([np.sqrt(7)/100, np.cos(2/3 * np.pi), np.sin(2/3 * np.pi)])
        positions[0,4,:] = np.asarray([np.sqrt(11)/100, np.cos(2/3 * np.pi), np.sin(4/3 * np.pi)])
        positions[0,5,:] = np.asarray([-11.,1.,np.sqrt(13)/100])
        positions[0,6,:] = np.asarray([-11., np.cos(2/3 * np.pi),np.sin(2/3 * np.pi)])
        positions[0,7,:] = np.asarray([-11.,np.cos(2/3 * np.pi), np.sin(4/3 * np.pi)])
        angles1 = np.tile(np.linspace(start=0., stop=2 * math.pi, num=int(np.sqrt(n)), endpoint=False), int(np.sqrt(n)))
        angles2 = np.repeat(np.linspace(start=0., stop=2 * math.pi, num=int(np.sqrt(n)), endpoint=False),
                            int(np.sqrt(n)))
        for i in range(1, n):
            #     rotationmatrix1 = np.zeros((3,3))
            #     rotationmatrix1[0,0] = 1
            #     rotationmatrix1[1,1] = np.cos(angles1[i])
            #     rotationmatrix1[1,2] = -np.sin(angles1[i])
            #     rotationmatrix1[2,2] = np.cos(angles1[i])
            #     rotationmatrix1[2,1] = np.sin(angles1[i])

            rotationmatrix1 = np.zeros((3, 3))
            rotationmatrix1[1, 1] = 1
            rotationmatrix1[0, 0] = np.cos(angles1[i])
            rotationmatrix1[0, 2] = -np.sin(angles1[i])
            rotationmatrix1[2, 2] = np.cos(angles1[i])
            rotationmatrix1[2, 0] = np.sin(angles1[i])

            rotationmatrix2 = np.zeros((3, 3))
            rotationmatrix2[0, 0] = 1
            rotationmatrix2[1, 1] = np.cos(angles2[i])
            rotationmatrix2[1, 2] = -np.sin(angles2[i])
            rotationmatrix2[2, 2] = np.cos(angles2[i])
            rotationmatrix2[2, 1] = np.sin(angles2[i])
            positions[i, np.asarray([3, 4]), :] = positions[0, np.asarray([3, 4]), :]
            positions[i, np.asarray([2, 8]), :] = np.matmul(rotationmatrix1,
                                                            positions[0, np.asarray([2, 8]), :].transpose()).transpose()
            positions[i, np.asarray([1, 5, 6, 7]), :] = np.matmul(rotationmatrix2,
                                                                  positions[0, np.asarray([1, 5, 6, 7]),
                                                                  :].transpose()).transpose()

        print(positions[0])
        covariance = np.identity(natoms)
        for i in range(natoms):
            for j in range(natoms):
                if i != j:
                    covariance[i, j] = cor
        # covariance = xvar * covariance
        # for i in range(n):
        #    for j in range(3):
        #        positions[i,:,j] = np.random.multivariate_normal(positions[i,:,j], covariance,size = 1)
        self.positions = positions

        print(positions[0])
        if angles == True:
            p = Pool(cores)
            results = p.map(lambda i: compute3angles(position=positions[i[0], atoms3[i[1]], :]), data_stream(10000, 84))
            data = np.reshape(results, (n, (d)))
        else:
            data = np.reshape(positions, (n, d))
        return (RiemannianManifold(data, dim))


    def get_dx_g_full(self, data):

        n = data.shape[0]
        d = self.d
        p = self.p
        output = np.zeros((n, p, d))
        for i in range(n):
            # if angles == True:
            #     pass
            # else:
            output[i, :, :] = self.get_dx_g_xyz(data[i]).transpose()
        return (output)

    #     def get_dx_g_pos_pytorch_full(self,data):
    #         n = data.shape[0]
    #         d = self.d
    #         p = self.p
    #         output = np.zeros((n,p,d))
    #         for i in range(n):
    #             output[i,:,:] = self.get_dx_g_pos_pytorch(data[i]).transpose()
    #         return(output)
    def get_g(self, x):
        atoms4 = self.atoms4
        # atoms3 = self.atoms3
        # p = len(atoms4)
        p = self.p
        # combos = np.asarray([[0,1,2],[1,2,3],[0,2,3],[0,1,3]])
        # d = atoms3.shape[0] * 3
        d = self.d
        output = np.zeros((p))
        # loop over tetrehedra
        for k in range(p):
            atom4 = atoms4[k, :]
            x4 = np.zeros((4, 3))
            for i in range(4):
                x4[i, :] = x[(atom4[i] * 3):(atom4[i] * 3 + 3)]
            x4 = np.asarray(x4)
            fitin = self.g4(x4, True)
            output[k] = fitin[0]
        return (output)

    def get_g_full(self, data):
        n = data.shape[0]
        p = self.p
        output = np.zeros((n, p))
        for i in range(n):
            # if angles == True:
            #     pass
            # else:
            output[i, :] = self.get_g(data[i]).transpose()
        return (output)

    def get_dx_g_xyz(self, x):
        atoms4 = self.atoms4
        # atoms3 = self.atoms3
        # p = len(atoms4)
        p = self.p
        # combos = np.asarray([[0,1,2],[1,2,3],[0,2,3],[0,1,3]])
        # d = atoms3.shape[0] * 3
        d = self.d
        output = np.zeros((d, p))
        # loop over tetrehedra
        for k in range(p):
            atom4 = atoms4[k, :]
            x4 = np.zeros((4, 3))
            for i in range(4):
                x4[i, :] = x[(atom4[i] * 3):(atom4[i] * 3 + 3)]
            x4 = np.asarray(x4)
            fitin = self.g4(x4, True)
            for i in range(4):
                # a = atoms3[actived[i]]
                for j in range(3):
                    # plus the lowest index first
                    output[3 * atom4[i] + j, k] = fitin[1][i, j]
        return (output)

    # need to check range of arccos
    def positions_to_torsion(self, positions4):
        positions4 = torch.tensor(positions4, requires_grad=True)
        d1 = positions4[0]
        c1 = positions4[1]
        c2 = positions4[2]
        d2 = positions4[3]
        cc = c2 - c1
        ip = torch.sum((d1 - c1) * (c2 - c1)) / (torch.sum((c2 - c1) ** 2))
        tilded1 = [d1[0] - ip * cc[0], d1[1] - ip * cc[1], d1[2] - ip * cc[2]]
        iq = torch.sum((d2 - c2) * (c1 - c2)) / (torch.sum((c1 - c2) ** 2))
        cc2 = (c1 - c2)
        tilded2 = [d2[0] - iq * cc2[0], d2[1] - iq * cc2[1], d2[2] - iq * cc2[2]]
        tilded2star = [tilded2[0] + cc2[0], tilded2[1] + cc2[1], tilded2[2] + cc2[2]]
        ab = torch.sqrt((tilded2star[0] - c1[0]) ** 2 + (tilded2star[1] - c1[1]) ** 2 + (tilded2star[2] - c1[2]) ** 2)
        bc = torch.sqrt((tilded2star[0] - tilded1[0]) ** 2 + (tilded2star[1] - tilded1[1]) ** 2 + (
                    tilded2star[2] - tilded1[2]) ** 2)
        ca = torch.sqrt((tilded1[0] - c1[0]) ** 2 + (tilded1[1] - c1[1]) ** 2 + (tilded1[2] - c1[2]) ** 2)
        output = torch.acos((ab ** 2 - bc ** 2 + ca ** 2) / (2 * ab * ca))
        return (output)

    def g4(self, positions4, grad=True):
        positions4 = torch.tensor(positions4, requires_grad=True)
        torsion = self.positions_to_torsion(positions4)
        torsion.backward(retain_graph=True)
        return (torsion, positions4.grad)


In [2]:
#this is for use with full dictionaries
#all points are members of a different irrational field
#in order to avoid set of measure zero
#in the noiseless case
#from codes.experimentclasses.AtomicRegression import AtomicRegression
from codes.geometer.RiemannianManifold import RiemannianManifold
from codes.otherfunctions.data_stream import data_stream
from codes.geometer.ShapeSpace import compute3angles
import numpy as np
import math
from sklearn.decomposition import TruncatedSVD

class RigidEthanolPCA3(AtomicRegression):
    """
    Parameters
    ----------
    cor : string,
        Data file to load
    xvar : np.array(dtype = int),
        List of adjacencies
    jj : np.array,
        List of adjacencies part 2
    d : int,
        dimension over which to evaluate the radii (smaller usually better)
    rmin : float,
        smallest radius ( = rad_bw_ratio * bandwidth) to consider
    rmax : float,
        largest radius ( = rad_bw_ratio * bandwidth) to consider
    ntry : int,
        number of radii between rmax and rmin to try
    run_parallel : bool,
        whether to run the analysis in parallel over radii
    search_space : str,
        either 'linspace' or 'logspace', choose to search in log or linear space
    rad_bw_ratio : float,
        the ratio of radius and kernel bandwidth, default to be 3 (radius = 3*h)
    Methods
    -------
    generate_data :
        Simulates data
    get_atoms_4 :
    	Gets atomic tetrahedra based off of ii and jj
    get_atoms_3 :
    	Gets triples of atoms
    """

    # AtomicRegression(dim, ii, jj, filename)
    def __init__(self, dim, cor, xvar,ii,jj, cores, noise, custom_bonds = None):
        natoms = 9
        n = 10000
        self.cor = cor
        self.xvar = xvar
        self.cores = cores
        self.noise = noise
        AtomicRegression.__init__(self, dim,n,ii,jj, natoms, cores)
        if custom_bonds.any() != None:
            self.atoms4 = custom_bonds
            self.p = custom_bonds.shape[0]

    def generate_data(self, noise=False):
        n = self.n
        d = self.d
        cor = self.cor
        xvar = self.xvar
        natoms = self.natoms
        cores = self.cores
        atoms3 = self.atoms3
        dim = self.dim
        #noise = self.noise

        positions = np.zeros((n, 9, 3))
        # positions[0,0,:] = np.asarray([0.,0.,0.])
        # positions[0,1,:] = np.asarray([-10.,0.,0.])
        # positions[0,2,:] = np.asarray([1.,10.,0.])
        # #positions[0,8,:] = np.asarray([1.,1.,0.])
        # positions[0,8,:] = np.asarray([2.,10.,-0])
        # positions[0,3,:] = np.asarray([1., np.cos(2/3 * np.pi), np.sin(2/3 * np.pi)])
        # positions[0,4,:] = np.asarray([1., np.cos(2/3 * np.pi), np.sin(4/3 * np.pi)])
        # positions[0,5,:] = np.asarray([-12.,1.,0.])
        # positions[0,6,:] = np.asarray([-12., np.cos(2/3 * np.pi),np.sin(2/3 * np.pi)])
        # positions[0,7,:] = np.asarray([-12.,np.cos(2/3 * np.pi), np.sin(4/3 * np.pi)])
        positions[0,0,:] = np.asarray([0.,0.,0.])
        positions[0,1,:] = np.asarray([-10.,0.,np.sqrt(2)/100])
        positions[0,2,:] = np.asarray([0.,10.,np.sqrt(3)/100])
        #positions[0,8,:] = np.asarray([1.,1.,0.])
        positions[0,8,:] = np.asarray([1.,10.,np.sqrt(5)/100])
        positions[0,3,:] = np.asarray([np.sqrt(7)/100, np.cos(2/3 * np.pi), np.sin(2/3 * np.pi)])
        positions[0,4,:] = np.asarray([np.sqrt(11)/100, np.cos(2/3 * np.pi), np.sin(4/3 * np.pi)])
        positions[0,5,:] = np.asarray([-11.,1.,np.sqrt(13)/100])
        positions[0,6,:] = np.asarray([-11. , np.cos(2/3 * np.pi)+ np.sqrt(17)/1000,np.sin(2/3 * np.pi) ])
        positions[0,7,:] = np.asarray([-11., np.cos(2/3 * np.pi)+ np.sqrt(19)/100, np.sin(4/3 * np.pi)])

        angles1 = np.tile(np.linspace(start=0., stop=2 * math.pi, num=int(np.sqrt(n)), endpoint=False),
                          int(np.sqrt(n)))
        angles2 = np.repeat(np.linspace(start=0., stop=2 * math.pi, num=int(np.sqrt(n)), endpoint=False),
                            int(np.sqrt(n)))
        for i in range(1, n):
            rotationmatrix1 = np.zeros((3, 3))
            rotationmatrix1[1, 1] = 1
            rotationmatrix1[0, 0] = np.cos(angles1[i])
            rotationmatrix1[0, 2] = -np.sin(angles1[i])
            rotationmatrix1[2, 2] = np.cos(angles1[i])
            rotationmatrix1[2, 0] = np.sin(angles1[i])
            rotationmatrix2 = np.zeros((3, 3))
            rotationmatrix2[0, 0] = 1
            rotationmatrix2[1, 1] = np.cos(angles2[i])
            rotationmatrix2[1, 2] = -np.sin(angles2[i])
            rotationmatrix2[2, 2] = np.cos(angles2[i])
            rotationmatrix2[2, 1] = np.sin(angles2[i])
            positions[i, np.asarray([3, 4]), :] = positions[0, np.asarray([3, 4]), :]
            positions[i, np.asarray([2, 8]), :] = np.matmul(rotationmatrix1,
                                                            positions[0, np.asarray([2, 8]),
                                                            :].transpose()).transpose()
            positions[i, np.asarray([1, 5, 6, 7]), :] = np.matmul(rotationmatrix2,
                                                                  positions[0, np.asarray([1, 5, 6, 7]),
                                                                  :].transpose()).transpose()

        covariance = np.identity(natoms)
        for i in range(natoms):
            for j in range(natoms):
                if i != j:
                    covariance[i, j] = cor
        covariance = xvar * covariance
        if noise == True:
            for i in range(n):
                for j in range(3):
                    positions[i, :, j] = np.random.multivariate_normal(positions[i, :, j], covariance, size=1)
        self.positions = positions
        p = Pool(cores)
        results = p.map(lambda i: compute3angles(position=positions[i[0], atoms3[i[1]], :]),
                            data_stream(10000, 84))
        data = np.reshape(results, (n, (d)))
        svd = TruncatedSVD(n_components=50)
        svd.fit(data)
        data_pca = svd.transform(data)
        return (RiemannianManifold(data, dim), RiemannianManifold(data_pca,dim), svd.components_)


def get_rigid_ethanol_data(cor, xvar, cores, atoms3, noise = False):

    n = 10000
    #d = self.d
    natoms = 9
    d = 252

    positions = np.zeros((n, 9, 3))
    positions[0, 0, :] = np.asarray([0., 0., 0.])
    positions[0, 1, :] = np.asarray([-10., 0., np.sqrt(2) / 100])
    positions[0, 2, :] = np.asarray([0., 10., np.sqrt(3) / 100])
    positions[0, 8, :] = np.asarray([1., 10., np.sqrt(5) / 100])
    positions[0, 3, :] = np.asarray([np.sqrt(7) / 100, np.cos(2 / 3 * np.pi), np.sin(2 / 3 * np.pi)])
    positions[0, 4, :] = np.asarray([np.sqrt(11) / 100, np.cos(2 / 3 * np.pi), np.sin(4 / 3 * np.pi)])
    positions[0, 5, :] = np.asarray([-11., 1., np.sqrt(13) / 100])
    positions[0, 6, :] = np.asarray([-11., np.cos(2 / 3 * np.pi) + np.sqrt(17) / 1000, np.sin(2 / 3 * np.pi)])
    positions[0, 7, :] = np.asarray([-11., np.cos(2 / 3 * np.pi) + np.sqrt(19) / 100, np.sin(4 / 3 * np.pi)])

    angles1 = np.tile(np.linspace(start=0., stop=2 * math.pi, num=int(np.sqrt(n)), endpoint=False),
                      int(np.sqrt(n)))
    angles2 = np.repeat(np.linspace(start=0., stop=2 * math.pi, num=int(np.sqrt(n)), endpoint=False),
                        int(np.sqrt(n)))
    for i in range(1, n):
        rotationmatrix1 = np.zeros((3, 3))
        rotationmatrix1[1, 1] = 1
        rotationmatrix1[0, 0] = np.cos(angles1[i])
        rotationmatrix1[0, 2] = -np.sin(angles1[i])
        rotationmatrix1[2, 2] = np.cos(angles1[i])
        rotationmatrix1[2, 0] = np.sin(angles1[i])
        rotationmatrix2 = np.zeros((3, 3))
        rotationmatrix2[0, 0] = 1
        rotationmatrix2[1, 1] = np.cos(angles2[i])
        rotationmatrix2[1, 2] = -np.sin(angles2[i])
        rotationmatrix2[2, 2] = np.cos(angles2[i])
        rotationmatrix2[2, 1] = np.sin(angles2[i])
        positions[i, np.asarray([3, 4]), :] = positions[0, np.asarray([3, 4]), :]
        positions[i, np.asarray([2, 8]), :] = np.matmul(rotationmatrix1,
                                                        positions[0, np.asarray([2, 8]),
                                                        :].transpose()).transpose()
        positions[i, np.asarray([1, 5, 6, 7]), :] = np.matmul(rotationmatrix2,
                                                              positions[0, np.asarray([1, 5, 6, 7]),
                                                              :].transpose()).transpose()

    covariance = np.identity(natoms)
    for i in range(natoms):
        for j in range(natoms):
            if i != j:
                covariance[i, j] = cor
    covariance = xvar * covariance
    if noise == True:
        for i in range(n):
            for j in range(3):
                positions[i, :, j] = np.random.multivariate_normal(positions[i, :, j], covariance, size=1)
    #self.positions = positions
    p = Pool(cores)
    results = p.map(lambda i: compute3angles(position=positions[i[0], atoms3[i[1]], :]),
                    data_stream(10000, 84))
    data = np.reshape(results, (n, (d)))
    #print(data.shape)
    #svd = TruncatedSVD(n_components=50)
    #svd.fit(data)
    #data_pca = svd.transform(data)
    return (data, positions)


In [3]:
import numpy as np

import torch
from codes.geometer.RiemannianManifold import RiemannianManifold
from codes.otherfunctions.data_stream_custom_range import data_stream_custom_range

#from code.source.utilities import data_stream_custom_range

class ShapeSpace(RiemannianManifold):

    def __init__(self, positions, angular_coordinates):
        self.positions = positions
        self.angular_coordinates = angular_coordinates

    def torchComputeAngle(self, poses):
        combos = torch.tensor([[0, 1], [1, 2], [2, 0]])
        ab = torch.norm(poses[combos[0, 0], :] - poses[combos[0, 1], :])
        bc = torch.norm(poses[combos[1, 0], :] - poses[combos[1, 1], :])
        ca = torch.norm(poses[combos[2, 0], :] - poses[combos[2, 1], :])
        output = torch.acos((ab ** 2 - bc ** 2 + ca ** 2) / (2 * ab * ca))
        return (output)

    def torchCompute3angles(self, position):
        angles = np.ones(3)
        gradients = np.zeros((3, position.shape[0], 3))
        for i in range(3):
            #print(i)
            poses = torch.tensor(position[[i, (i + 1) % 3, (i + 2) % 3], :], requires_grad=True)
            tempang = self.torchComputeAngle(poses)
            tempang.backward()
            angles[i] = tempang.detach().numpy()
            gradients[i] = poses.grad[[(2 * i) % 3, (2 * i + 1) % 3, (2 * i + 2) % 3], :]
            # del(poses)
        return(angles, gradients)

    def reshapepointdata(self, pointdata, atoms3):
        natoms = len(np.unique(atoms3))
        output = np.zeros((pointdata.shape[0] * pointdata.shape[1], natoms * 3))
        for i in range(pointdata.shape[0]):
            for j in range(pointdata.shape[1]):
                for k in range(pointdata.shape[2]):
                    for l in range(pointdata.shape[3]):
                        # print(atoms3[k]*3 + l)
                        output[i * 3 + j, atoms3[i][k] * 3 + l] = pointdata[i, j, k, l]
        return(output)

    def get_wilson(self, selind, atoms3, tdata):
        natoms = len(np.unique(atoms3))
        jacobien = np.zeros((len(selind), len(atoms3) * 3, natoms * 3))
        for i in range(len(selind)):
            pointdata = tdata[i * len(atoms3): (i + 1) * len(atoms3)]
            jacobien[i] = self.reshapepointdata(pointdata, atoms3)
        return(jacobien)

    def get_internal_projector(self, natoms, jacobien, selind):
        nnonzerosvd = 3 * natoms - 7
        internalprojector = np.zeros((len(selind), jacobien.shape[1], nnonzerosvd))
        for i in range(len(selind)):
            asdf = np.linalg.svd(jacobien[i])
            internalprojector[i] = (asdf[0][:, :nnonzerosvd] )
        return (internalprojector)

    def get_dw(self,cores,atoms3,natoms, selected_points):
        positions = self.positions
        self.selected_points = selected_points
        p = Pool(cores)
        n = len(selected_points)
        results = p.map(lambda i: self.torchCompute3angles(position=positions[i[0], atoms3[i[1]], :]),
                        data_stream_custom_range(selected_points, len(atoms3)))
        tdata = np.asarray([results[i][1] for i in range(n * len(atoms3))])
        # for i in range(len(selected_points)):
        #     pointdata = tdata[i * len(atoms3): (i + 1) * len(atoms3)]
        jacobien = self.get_wilson(selected_points, atoms3, tdata)
        internalprojector = self.get_internal_projector(natoms, jacobien, selected_points)
        return(internalprojector)


def torchComputeAngle(poses):
    combos = torch.tensor([[0, 1], [1, 2], [2, 0]])
    ab = torch.norm(poses[combos[0, 0], :] - poses[combos[0, 1], :])
    bc = torch.norm(poses[combos[1, 0], :] - poses[combos[1, 1], :])
    ca = torch.norm(poses[combos[2, 0], :] - poses[combos[2, 1], :])
    output = torch.acos((ab ** 2 - bc ** 2 + ca ** 2) / (2 * ab * ca))
    return (output)
def torchCompute3angles(position):
    angles = np.ones(3)
    gradients = np.zeros((3, position.shape[0], 3))
    for i in range(3):
        #print(i)
        poses = torch.tensor(position[[i, (i + 1) % 3, (i + 2) % 3], :], requires_grad=True)
        tempang = torchComputeAngle(poses)
        tempang.backward()
        angles[i] = tempang.detach().numpy()
        gradients[i] = poses.grad[[(2 * i) % 3, (2 * i + 1) % 3, (2 * i + 2) % 3], :]
        # del(poses)
    return(angles, gradients)

In [4]:

n = 10000  # number of data points to simulate
nsel = 100  # number of points to analyze with lasso
n_neighbors = 1000  # number of neighbors in megaman
m = 3  # number of embedding dimensions (diffusion maps)
diffusion_time = 0.05  # embedding radius
dim = 2  # manifold dimension
dimnoise = 2  # manifold dimension (not in mflasso paper)
cores = 3
nreps = 1
natoms = 9
ii = np.asarray([0, 0, 0, 0, 1, 1, 1, 2])  # atom adjacencies for dihedral angle computation
jj = np.asarray([1, 2, 3, 4, 5, 6, 7, 8])

savename = 'rigidethanol_021521'
savefolder = 'rigidethanol'
# loadfolder = 'rigidethanol'
# loadname = 'ethanol_021521'
#data_wd = '/Users/samsonkoelle/Downloads/manigrad-100818/mani-samk-gradients/'

#folder = workingdirectory + '/Figures/rigidethanol/' + now + 'n' + str(n) + 'nsel' + str(nsel) + 'nreps' + str(nreps)
#os.mkdir(folder)
experiment = AtomicRegression(natoms)
experiment.m = m
experiment.dim = dim
experiment.dnoise = dim
experiment.nreps = nreps
experiment.nsel = nsel
experiment.atoms3, experiment.da = experiment.get_atoms_3()
experiment.atoms4, experiment.p = get_atoms_4(natoms, ii, jj)

#data, experiment.positions = get_rigid_ethanol_data(cor=0., xvar=0., cores=3, atoms3=experiment.atoms3, noise=False)


In [5]:
from pathos.multiprocessing import ProcessingPool as Pool
#from pathos.multiprocessing import ProcessingPool as Pool
#from pathos.multiprocessing import ProcessingPool as Pool
#import math

In [6]:
    #def get_rigid_ethanol_data(cor, xvar, cores, atoms3, noise = False):abs
    cor = 0.
    atoms3 = experiment.atoms3
    n = 10000
    #d = self.d
    natoms = 9
    d = 252
    xvar = 0.
    noise= False
    positions = np.zeros((n, 9, 3))
    positions[0, 0, :] = np.asarray([0., 0., 0.])
    positions[0, 1, :] = np.asarray([-10., 0., np.sqrt(2) / 100])
    positions[0, 2, :] = np.asarray([0., 10., np.sqrt(3) / 100])
    positions[0, 8, :] = np.asarray([1., 10., np.sqrt(5) / 100])
    positions[0, 3, :] = np.asarray([np.sqrt(7) / 100, np.cos(2 / 3 * np.pi), np.sin(2 / 3 * np.pi)])
    positions[0, 4, :] = np.asarray([np.sqrt(11) / 100, np.cos(2 / 3 * np.pi), np.sin(4 / 3 * np.pi)])
    positions[0, 5, :] = np.asarray([-11., 1., np.sqrt(13) / 100])
    positions[0, 6, :] = np.asarray([-11., np.cos(2 / 3 * np.pi) + np.sqrt(17) / 1000, np.sin(2 / 3 * np.pi)])
    positions[0, 7, :] = np.asarray([-11., np.cos(2 / 3 * np.pi) + np.sqrt(19) / 100, np.sin(4 / 3 * np.pi)])

    angles1 = np.tile(np.linspace(start=0., stop=2 * math.pi, num=int(np.sqrt(n)), endpoint=False),
                      int(np.sqrt(n)))
    angles2 = np.repeat(np.linspace(start=0., stop=2 * math.pi, num=int(np.sqrt(n)), endpoint=False),
                        int(np.sqrt(n)))
    for i in range(1, n):
        rotationmatrix1 = np.zeros((3, 3))
        rotationmatrix1[1, 1] = 1
        rotationmatrix1[0, 0] = np.cos(angles1[i])
        rotationmatrix1[0, 2] = -np.sin(angles1[i])
        rotationmatrix1[2, 2] = np.cos(angles1[i])
        rotationmatrix1[2, 0] = np.sin(angles1[i])
        rotationmatrix2 = np.zeros((3, 3))
        rotationmatrix2[0, 0] = 1
        rotationmatrix2[1, 1] = np.cos(angles2[i])
        rotationmatrix2[1, 2] = -np.sin(angles2[i])
        rotationmatrix2[2, 2] = np.cos(angles2[i])
        rotationmatrix2[2, 1] = np.sin(angles2[i])
        positions[i, np.asarray([3, 4]), :] = positions[0, np.asarray([3, 4]), :]
        positions[i, np.asarray([2, 8]), :] = np.matmul(rotationmatrix1,
                                                        positions[0, np.asarray([2, 8]),
                                                        :].transpose()).transpose()
        positions[i, np.asarray([1, 5, 6, 7]), :] = np.matmul(rotationmatrix2,
                                                              positions[0, np.asarray([1, 5, 6, 7]),
                                                              :].transpose()).transpose()

    covariance = np.identity(natoms)
    for i in range(natoms):
        for j in range(natoms):
            if i != j:
                covariance[i, j] = cor
    covariance = xvar * covariance
    if noise == True:
        for i in range(n):
            for j in range(3):
                positions[i, :, j] = np.random.multivariate_normal(positions[i, :, j], covariance, size=1)
    #self.positions = positions
    p = Pool(cores)
    #print(data.shape)
    #svd = TruncatedSVD(n_components=50)
    #svd.fit(data)
    #data_pca = svd.transform(data)
    #return (data, positions)

Process ForkPoolWorker-2:
Process ForkPoolWorker-3:
Process ForkPoolWorker-1:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/samsonkoelle/anaconda3/envs/manifold_env_april/lib/python3.5/site-packages/multiprocess/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/Users/samsonkoelle/anaconda3/envs/manifold_env_april/lib/python3.5/site-packages/multiprocess/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/samsonkoelle/anaconda3/envs/manifold_env_april/lib/python3.5/site-packages/multiprocess/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/samsonkoelle/anaconda3/envs/manifold_env_april/lib/python3.5/site-packages/multiprocess/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/samsonkoelle/anaconda3/envs/manifold_env_april/lib/python3.5/site-packages/multiprocess/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))


In [8]:
    results = p.map(lambda i: compute3angles(position=positions[i[0], atoms3[i[1]], :]),
                    data_stream(10000, 84))
    data = np.reshape(results, (n, (d)))


In [11]:
torchCompute3angles(position=positions[0, atoms3[0], :])

(array([1.57079388, 0.78539964, 0.78539914]),
 array([[[-9.99995051e-02,  9.99995051e-02,  3.14624880e-04],
         [-2.44947872e-07, -9.99997500e-02, -1.73204301e-04],
         [ 9.99997500e-02,  2.44947627e-07, -1.41420578e-04]],
 
        [[-2.44947872e-07, -9.99997500e-02, -1.73204301e-04],
         [ 5.00001449e-02,  4.99999000e-02,  1.58914841e-05],
         [-4.99999000e-02,  4.99998500e-02,  1.57312817e-04]],
 
        [[ 9.99997500e-02,  2.44947627e-07, -1.41420578e-04],
         [-4.99999000e-02,  4.99998500e-02,  1.57312817e-04],
         [-4.99998500e-02, -5.00000949e-02, -1.58922388e-05]]]))

In [17]:
    results = p.map(lambda i: torchCompute3angles(position=positions[i[0], atoms3[i[1]], :]),
                    data_stream(100, 84))


In [20]:
results[0][1].shape

(3, 3, 3)

In [21]:
import sys
!{sys.executable} -m pip install numba

In [78]:
from numba import njit, prange

@njit(parallel=True)
def prange_test(A):
    s = 0
    # Without "parallel=True" in the jit-decorator
    # the prange statement is equivalent to range
    for i in prange(A.shape[0]):
        s += A[i]
    return s

In [24]:
prange_test(np.asarray(list(range(100))))

4950

In [25]:
    A = np.asarray(list(range(100)))
    s = 0
    # Without "parallel=True" in the jit-decorator
    # the prange statement is equivalent to range
    for i in prange(A.shape[0]):
        s += A[i]


In [26]:
s

4950

In [27]:
positions

array([[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-1.00000000e+01,  0.00000000e+00,  1.41421356e-02],
        [ 0.00000000e+00,  1.00000000e+01,  1.73205081e-02],
        ...,
        [-1.10000000e+01, -4.95876894e-01,  8.66025404e-01],
        [-1.10000000e+01, -4.56411011e-01, -8.66025404e-01],
        [ 1.00000000e+00,  1.00000000e+01,  2.23606798e-02]],

       [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-1.00000000e+01,  0.00000000e+00,  1.41421356e-02],
        [-1.08756370e-03,  1.00000000e+01,  1.72863300e-02],
        ...,
        [-1.10000000e+01, -4.95876894e-01,  8.66025404e-01],
        [-1.10000000e+01, -4.56411011e-01, -8.66025404e-01],
        [ 9.96622690e-01,  1.00000000e+01,  8.51070756e-02]],

       [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-1.00000000e+01,  0.00000000e+00,  1.41421356e-02],
        [-2.17083528e-03,  1.00000000e+01,  1.71839307e-02],
        ...,
        [-1.10000000e+01, -4.95876894e-01,

In [68]:
from numba import jit

In [245]:
@jit(nopython=True)
def computeAngle(poses):
    combos = np.array([[0,1],[1,2],[2,0]])
    ab = np.linalg.norm(poses[combos[0,0],:] - poses[combos[0,1],:])
    bc = np.linalg.norm(poses[combos[1,0],:] - poses[combos[1,1],:])
    ca = np.linalg.norm(poses[combos[2,0],:] - poses[combos[2,1],:])
    output = np.arccos((ab**2 - bc**2 + ca**2) / (2 * ab * ca))
    #output = (ab**2 - bc**2 + ca**2) / (2 * ab * ca)
    return(output)

@jit(nopython=True)
def compute3angles(position):
    angles = np.ones(3)
    for i in range(3):
        #poses = position[[i, (i+1) %3, (i+2) % 3],:]
        #inds = 
        2+2
        #print(position.shape)
        #position[i]
        #position[[i, (i+1) %3, (i+2) % 3],:]
        #print(position.shape)
        position[0]
        #position.take([i, (i+1) %3, (i+2) % 3], axis = 1)
        #position.take([i, (i+1) %3, (i+2) % 3], axis = 1)
        #angles[i] = computeAngle()
    return(angles)

In [246]:
#positions[0, atoms3[j], :][[i, (i+1) %3, (i+2) % 3],:]
#positions[0, atoms3[j], :].take([i, (i+1) %3, (i+2) % 3], axis = 1)

In [247]:
i = 0 
computeAngle(positions[0, atoms3[j], :].take([i, (i+1) %3, (i+2) % 3], axis = 1))

0.15330509610088225

In [248]:
compute3angles(position=positions[0, atoms3[j], :])

array([1., 1., 1.])

In [79]:
@njit(parallel=True)
def prange_test(positions, atoms3):
    #atoms3 = experiment.atoms3
    results = np.zeros((positions.shape[0],atoms3.shape[0]*3))
    i  =0 
    #for i in prange(10000):
    for j in prange(84):
        results[0,(j*3):(j*3 + 3)] = compute3angles(position=positions[0, atoms3[j], :])
    return results

In [80]:
prange_test(positions, atoms3)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Internal error at <numba.typeinfer.IntrinsicCallConstraint object at 0x12ec4ec18>:
--%<----------------------------------------------------------------------------
Traceback (most recent call last):
  File "/Users/samsonkoelle/anaconda3/envs/manifold_env_april/lib/python3.5/site-packages/numba/errors.py", line 599, in new_error_context
    yield
  File "/Users/samsonkoelle/anaconda3/envs/manifold_env_april/lib/python3.5/site-packages/numba/typeinfer.py", line 513, in __call__
    self.resolve(typeinfer, typeinfer.typevars, fnty=self.func)
  File "/Users/samsonkoelle/anaconda3/envs/manifold_env_april/lib/python3.5/site-packages/numba/typeinfer.py", line 450, in resolve
    literals=literals)
  File "/Users/samsonkoelle/anaconda3/envs/manifold_env_april/lib/python3.5/site-packages/numba/typeinfer.py", line 1173, in resolve_call
    literals=literals)
  File "/Users/samsonkoelle/anaconda3/envs/manifold_env_april/lib/p

In [63]:
#@njit(parallel=True)
def computeAngle(poses):
    combos = np.asarray([[0,1],[1,2],[2,0]])
    ab = np.linalg.norm(poses[combos[0,0],:] - poses[combos[0,1],:])
    bc = np.linalg.norm(poses[combos[1,0],:] - poses[combos[1,1],:])
    ca = np.linalg.norm(poses[combos[2,0],:] - poses[combos[2,1],:])
    output = np.arccos((ab**2 - bc**2 + ca**2) / (2 * ab * ca))
    #output = (ab**2 - bc**2 + ca**2) / (2 * ab * ca)
    return(output)

#@njit(parallel=True)
def compute3angles(position):
    angles = np.ones(3)
    for i in range(3):
        poses = position[[i, (i+1) %3, (i+2) % 3],:]
        angles[i] = computeAngle(poses)
    return(angles)

In [64]:
computeAngle(positions[0, atoms3[j], :])

0.15330509610088225

In [65]:
compute3angles(position=positions[0, atoms3[j], :])

array([0.1533051 , 1.49248098, 1.49580658])